# Исследование надежности заемщиков.

  Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.
  Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.


### Шаг 1. Обзор данных

Составим первое представление о данных статистики платёжеспособности клиентов.

In [1]:
import pandas as pd # импорт библиотеки pandas

In [2]:
df = pd.read_csv('/datasets/data.csv')  # чтение файла с данными и сохранение в df

In [3]:
df.head(10) # получение первых 10 строк таблицы df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [4]:
df.info()  # получение общей информации о данных в таблице df

<class 'pandas.core.frame.DataFrame'>

RangeIndex: 21525 entries, 0 to 21524

Data columns (total 12 columns):

 #   Column            Non-Null Count  Dtype  

---  ------            --------------  -----  

 0   children          21525 non-null  int64  

 1   days_employed     19351 non-null  float64

 2   dob_years         21525 non-null  int64  

 3   education         21525 non-null  object 

 4   education_id      21525 non-null  int64  

 5   family_status     21525 non-null  object 

 6   family_status_id  21525 non-null  int64  

 7   gender            21525 non-null  object 

 8   income_type       21525 non-null  object 

 9   debt              21525 non-null  int64  

 10  total_income      19351 non-null  float64

 11  purpose           21525 non-null  object 

dtypes: float64(2), int64(5), object(5)

memory usage: 2.0+ MB


В таблице 12 столбцов. Тип данных в столбцах разный — `object`, `int64`, `float64`.

Согласно документации к данным:
* `children ` — количество детей в семье;
* `days_employed` — общий трудовой стаж в днях;  
* `dob_years` — возраст клиента в годах;
* `education` — уровень образования клиента;
* `education_id` — идентификатор уровня образования;
* `family_status` — семейное положение;
* `family_status_id` — идентификатор семейного положения;
* `gender ` — пол клиента;
* `income_type ` — тип занятости ;
* `debt` — имел ли задолженность по возврату кредитов;
* `total_income` — ежемесячный доход;
* `purpose` — цель получения кредита.

Названия столбцов не содержат пробелов, все знаки в одном регистре и на одном языке. 
Cтолбец days_employed содержит нехарактерные для него значения (отрицательные), столбец education - данные, записанные в разном регистре(наличие дубликатов).
Количество значений в столбцах days_employed и total_income имеют одинаковые количество non-null.  Вероятно это те у кого еще нет стажа работы и соответственно они еще не зарабатывают. Необходимо будет выяснить возраст таких клиентов.
Типы столбцов соответствуют информации хранящихся в них.


### Шаг 2.1 Заполнение пропусков

In [5]:
#print(f'Процент отсутствующих значений = {(21525-19351)/21525:,.2%}')

In [6]:
df.isna().mean()

children            0.000000
days_employed       0.100999
dob_years           0.000000
education           0.000000
education_id        0.000000
family_status       0.000000
family_status_id    0.000000
gender              0.000000
income_type         0.000000
debt                0.000000
total_income        0.100999
purpose             0.000000
dtype: float64

Доля пропущенных значений в каждом из столбцов с пропусками составляет 10.1%. Это довольно большой процент, значение которого нужно исправлять.

Мы обнаружили явные пропуски в данных столбцах. Теперь проверим столбцы и разберемся в причине их возникновения

Столбец 'days_employed'

In [7]:
print(df['days_employed'])
print()
print("Отрицательные значения = ",df[df['days_employed']<0]['days_employed'].count())
print()
days_employed_median = df['days_employed'].median()
print(days_employed_median)

0         -8437.673028

1         -4024.803754

2         -5623.422610

3         -4124.747207

4        340266.072047

             ...      

21520     -4529.316663

21521    343937.404131

21522     -2113.346888

21523     -3112.481705

21524     -1984.507589

Name: days_employed, Length: 21525, dtype: float64



Отрицательные значения =  15906



-1203.369528770489


Из полученного результата мы видим, что присутствуют слишком большие значения столбца(неизвестны единицы измерения) и много отрицательных значений - оба варианта недопустимы.


In [8]:
#заменим отрицательные числа на положительные в стаже
def negative_to_pozitive(value):
    if value < 0:
        value *= -1
        return value
    else:
        return value

df['days_employed'] = df['days_employed'].apply(negative_to_pozitive)
df['days_employed'] = df['days_employed'].fillna(days_employed_median)
print(df['days_employed'])


0          8437.673028

1          4024.803754

2          5623.422610

3          4124.747207

4        340266.072047

             ...      

21520      4529.316663

21521    343937.404131

21522      2113.346888

21523      3112.481705

21524      1984.507589

Name: days_employed, Length: 21525, dtype: float64


В стаже отрицательные числа сделали положительными, пропуски заменили на медианное значение.
Если бы была возможность уточнить данные значения у заказчика, то можно было бы внести исправления. 

Столбец 'total_income'

In [9]:
print(df['total_income']) 
display(df[df['total_income'].isnull()].head(10))

0        253875.639453

1        112080.014102

2        145885.952297

3        267628.550329

4        158616.077870

             ...      

21520    224791.862382

21521    155999.806512

21522     89672.561153

21523    244093.050500

21524     82047.418899

Name: total_income, Length: 21525, dtype: float64


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,-1203.369529,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,-1203.369529,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,-1203.369529,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,-1203.369529,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,-1203.369529,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,-1203.369529,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,-1203.369529,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,-1203.369529,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,-1203.369529,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,-1203.369529,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


Мы вывели 10 строчек: по столбцу "dob_years" видно , что возраст варьируется от 21 до 65. Значит предположение ,что это никогда неработавшие - ложно. Во всех остальных столбцах тоже стоят разные значения. Значит будем заполнять 'total_income' средним значением (median).

In [10]:
total_income_median = df['total_income'].median() #найдем медиану дохода
print(total_income_median)
print()
df['total_income'] = df['total_income'].fillna(total_income_median)
print(df['total_income']) 

145017.93753253992



0        253875.639453

1        112080.014102

2        145885.952297

3        267628.550329

4        158616.077870

             ...      

21520    224791.862382

21521    155999.806512

21522     89672.561153

21523    244093.050500

21524     82047.418899

Name: total_income, Length: 21525, dtype: float64


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>

RangeIndex: 21525 entries, 0 to 21524

Data columns (total 12 columns):

 #   Column            Non-Null Count  Dtype  

---  ------            --------------  -----  

 0   children          21525 non-null  int64  

 1   days_employed     21525 non-null  float64

 2   dob_years         21525 non-null  int64  

 3   education         21525 non-null  object 

 4   education_id      21525 non-null  int64  

 5   family_status     21525 non-null  object 

 6   family_status_id  21525 non-null  int64  

 7   gender            21525 non-null  object 

 8   income_type       21525 non-null  object 

 9   debt              21525 non-null  int64  

 10  total_income      21525 non-null  float64

 11  purpose           21525 non-null  object 

dtypes: float64(2), int64(5), object(5)

memory usage: 2.0+ MB


В данных случаях заполнить пропуски медианными значениями лучше, т.к они лучше характеризует данные, когда некоторые значения сильно выделяются среди большинства.

В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. Для реальных данных это нормально. Посмотрим на данные других столбцов.

### Шаг 2.2 Проверка данных на аномалии и исправления.

Столбец 'children'

In [12]:
print(df['children'].value_counts())

 0     14149

 1      4818

 2      2055

 3       330

 20       76

-1        47

 4        41

 5         9

Name: children, dtype: int64


Самое большое количество клиентов бездетных, затем по уменьшению, кто имеет 1, 2 и более детей. Аномальными значения являются 20 детей (76) и -1 (47). Эти значения маловероятны, тем более отрицательные значения. Количество таких семей составляет 0.56% от общего количества. Скорее этими данными можно принебречь, удалив их.

In [13]:
i1 = df[df['children']==-1].index # индексы 47 строк где значение -1
print(i1)
i20 = df[df['children']==20].index # индексы 76 строк где значение 20 
print(i20)
print()
#Удаление аномальных значений
str_del_count=0
try:
    for drop_str in i1:
        df = df.drop(drop_str)
        str_del_count+=1
except:
    print('Невозможно удалить строку')
print('Кол-во удаленных строк c "-1" =', str_del_count)

str_del_count=0
try:
    for drop_str in i20:
        df = df.drop(drop_str)
        str_del_count+=1
except:
    print('Невозможно удалить строку')    
print('Кол-во удаленных строк c "20" =', str_del_count)
print()
#Проверяем 
print(df[df['children']==20].head())
print(df[df['children']==-1].head())
  
df = df.reset_index(drop=True)
df.info()


Int64Index([  291,   705,   742,   800,   941,  1363,  1929,  2073,  3814,

             4201,  4402,  4542,  5272,  6013,  6385,  7190,  7278,  7615,

             7685,  8229,  8251,  8561,  9565,  9574, 10177, 10365, 10918,

            11109, 11268, 11617, 12304, 13786, 14357, 15147, 15435, 16129,

            16265, 17095, 17429, 17657, 18219, 19100, 19366, 19417, 20393,

            20717, 21140],

           dtype='int64')

Int64Index([  606,   720,  1074,  2510,  2941,  3302,  3396,  3671,  3697,

             3735,  3877,  5020,  5315,  5349,  5362,  6198,  6428,  7027,

             7078,  7516,  7789,  7962,  8430,  8883,  8909,  8950,  9889,

            10194, 10377, 10736, 10782, 11449, 11489, 11715, 11937, 12212,

            12226, 12319, 12390, 12534, 12909, 12910, 13256, 13291, 13471,

            13489, 13618, 14355, 14474, 14782, 15313, 15674, 15762, 15812,

            15976, 16371, 16795, 16886, 17093, 17134, 17286, 17509, 18476,

            18493, 18756, 18890, 

In [14]:
pd.DataFrame({'Кол-во заёмщиков по количеству детей':df['children'].value_counts()}) #столбец с новыми значениями

,Кол-во заёмщиков по количеству детей
0,14149
1,4818
2,2055
3,330
4,41
5,9


Столбец 'dob_years'

In [16]:
print(df['dob_years'].value_counts().sort_index())

0     100

19     14

20     51

21    110

22    183

23    252

24    263

25    356

26    406

27    490

28    501

29    543

30    536

31    556

32    506

33    577

34    597

35    614

36    553

37    531

38    595

39    572

40    603

41    603

42    592

43    510

44    543

45    494

46    469

47    480

48    536

49    505

50    509

51    446

52    483

53    457

54    476

55    441

56    482

57    457

58    461

59    441

60    376

61    353

62    351

63    268

64    263

65    194

66    183

67    167

68     99

69     83

70     65

71     58

72     33

73      8

74      6

75      1

Name: dob_years, dtype: int64


Аномалией данного столбца является значение 0. Нужно проверить влияние данной аномалии и при возможности удалить, т.к процент таких значений равен всего 0.47 от общего количества.

In [17]:
display(df[df['dob_years']==0].head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,0,346541.618895,0,Среднее,1,женат / замужем,0,F,пенсионер,0,71291.522491,автомобиль
149,0,2664.273168,0,среднее,1,в разводе,3,F,сотрудник,0,70176.435951,операции с жильем
270,3,1872.663186,0,среднее,1,женат / замужем,0,F,сотрудник,0,102166.458894,ремонт жилью
577,0,397856.565013,0,среднее,1,женат / замужем,0,F,пенсионер,0,97620.687042,строительство собственной недвижимости
1033,0,1158.029561,0,высшее,0,в разводе,3,F,компаньон,0,303994.134987,свой автомобиль
1141,0,934.654854,0,среднее,1,женат / замужем,0,F,компаньон,0,201852.430096,покупка недвижимости
1167,0,370879.508002,0,среднее,1,женат / замужем,0,F,пенсионер,0,313949.845188,получение дополнительного образования
1377,0,5043.219890,0,высшее,0,женат / замужем,0,M,госслужащий,0,240523.618071,сделка с автомобилем
1881,0,-1203.369529,0,высшее,0,Не женат / не замужем,4,F,сотрудник,0,145017.937533,жилье
1889,0,370144.537021,0,среднее,1,вдовец / вдова,2,F,пенсионер,0,127400.268338,на покупку автомобиля


Закономерность не выявлена, поэтому удаляю эти строки.

In [18]:
i0 = df[df['dob_years']==0].index # индексы строк где значение 0 
print(i0, 'Кол-во =', len(i0))
print()
#Удаление аномальных значений 
str_del_count=0
for drop_str in i0:
    df = df.drop(drop_str)
    str_del_count+=1
print('Количество удаленных строк c "0" =', str_del_count)
print()
#Проверяем
print(df[df['dob_years']==0].head())
print()  
df = df.reset_index(drop=True)
print()  
df.info()

Int64Index([   99,   149,   270,   577,  1033,  1141,  1167,  1377,  1881,

             1889,  2071,  2273,  2458,  2476,  2858,  3031,  3134,  3687,

             4044,  4127,  4899,  4907,  4991,  6042,  6376,  6380,  6638,

             6746,  6799,  6816,  6827,  7001,  7040,  7042,  7217,  7308,

             8020,  8373,  8529,  8627,  8739,  9014,  9103,  9538, 10112,

            10136, 10253, 10311, 10329, 10490, 10540, 11019, 11229, 11407,

            11420, 11514, 11601, 11925, 11997, 12159, 12230, 12333, 12658,

            13044, 13114, 13159, 13338, 13348, 13364, 13444, 13663, 13889,

            14395, 14432, 14526, 14577, 15057, 15211, 15324, 15348, 15797,

            15802, 15952, 16767, 16827, 17117, 17207, 17512, 18074, 18312,

            18434, 18627, 18745, 19008, 19261, 19716, 20346, 20461, 21060,

            21194],

           dtype='int64') Кол-во = 100



Количество удаленных строк c "0" = 100



Empty DataFrame

Columns: [children, days_employed, dob_yea

Столбец 'education' 

In [19]:
print(df['education'].value_counts())

среднее                13609

высшее                  4666

СРЕДНЕЕ                  764

Среднее                  700

неоконченное высшее      663

ВЫСШЕЕ                   270

Высшее                   266

начальное                250

Неоконченное высшее       47

НЕОКОНЧЕННОЕ ВЫСШЕЕ       29

НАЧАЛЬНОЕ                 17

Начальное                 15

ученая степень             4

Ученая степень             1

УЧЕНАЯ СТЕПЕНЬ             1

Name: education, dtype: int64


 В столбце 'education' есть одиноковые записи, но с разными регистрами, надо эти дубликаты исправить.

Столбец 'education_id '

In [20]:
print(df['education_id'].value_counts())

1    15073

0     5202

2      739

3      282

4        6

Name: education_id, dtype: int64


По столбцу 'education_id' видно, что существует 5 типов образования, без аномалий.

Столбец 'family_status'

In [21]:
print(df['family_status'].value_counts())

женат / замужем          12254

гражданский брак          4139

Не женат / не замужем     2783

в разводе                 1179

вдовец / вдова             947

Name: family_status, dtype: int64


В данном столбце аномалий не наблюдается.

Столбец 'family_status_id'

In [22]:
print(df['family_status_id'].value_counts())

0    12254

1     4139

4     2783

3     1179

2      947

Name: family_status_id, dtype: int64


Значения столбца 'family_status_id'совпадают с значениями столбца 'family_status', без аномалий.

 Столбец 'gender '

In [23]:
print(df['gender'].value_counts())

F      14083

M       7218

XNA        1

Name: gender, dtype: int64


In [24]:
display(df.loc[df['gender']=='XNA'])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10595,0,2358.600502,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости


Всего только 1 строка с аномальным значением - 'XNA' - скорее всего это пропуск. Но мы не знаем точно, т.к не можем сделать предположений из данных. Это значение не является основным критерием для оценки, но строка эта важна. Поэтому оставляем без изменений.

Столбец 'income_type'

In [25]:
print(df['income_type'].value_counts())

сотрудник          10996

компаньон           5034

пенсионер           3819

госслужащий         1447

предприниматель        2

безработный            2

в декрете              1

студент                1

Name: income_type, dtype: int64


Столбец без аномалий.

Столбец 'debt'

In [26]:
print(df['debt'].value_counts())

0    19578

1     1724

Name: debt, dtype: int64


Данный столбец также без аномалий.

Столбец 'purpose'

In [27]:
print(df['purpose'].value_counts())

свадьба                                   791

на проведение свадьбы                     768

сыграть свадьбу                           764

операции с недвижимостью                  670

покупка коммерческой недвижимости         658

покупка жилья для сдачи                   649

операции с коммерческой недвижимостью     644

операции с жильем                         642

покупка жилья для семьи                   639

жилье                                     636

покупка жилья                             635

недвижимость                              628

операции со своей недвижимостью           626

строительство собственной недвижимости    626

строительство недвижимости                620

строительство жилой недвижимости          619

покупка своего жилья                      618

покупка недвижимости                      616

ремонт жилью                              607

покупка жилой недвижимости                600

на покупку своего автомобиля              501

заняться высш

Столбец корректный, много совпадающих целей кредита.Необходимо структурировать названия.

### Шаг 2.3. Изменение типов данных.

In [28]:
df.info() #до изменений
df['total_income'] = df['total_income'].astype(int)
print()
df.info() #после изменеий

<class 'pandas.core.frame.DataFrame'>

RangeIndex: 21302 entries, 0 to 21301

Data columns (total 12 columns):

 #   Column            Non-Null Count  Dtype  

---  ------            --------------  -----  

 0   children          21302 non-null  int64  

 1   days_employed     21302 non-null  float64

 2   dob_years         21302 non-null  int64  

 3   education         21302 non-null  object 

 4   education_id      21302 non-null  int64  

 5   family_status     21302 non-null  object 

 6   family_status_id  21302 non-null  int64  

 7   gender            21302 non-null  object 

 8   income_type       21302 non-null  object 

 9   debt              21302 non-null  int64  

 10  total_income      21302 non-null  float64

 11  purpose           21302 non-null  object 

dtypes: float64(2), int64(5), object(5)

memory usage: 2.0+ MB



<class 'pandas.core.frame.DataFrame'>

RangeIndex: 21302 entries, 0 to 21301

Data columns (total 12 columns):

 #   Column            Non-Null Count 

### Шаг 2.4. Удаление дубликатов.

Посчитаем количество дубликатов во всех столбцах. Подсчет дубликатов не применим к столбцам, либо дубликаты в них не обнаружены: days_employed, education_id, family_status, family_status_id, income_type, debt, total_income

Столбец 'education'

В столбце `education` необходимо обработать дубликаты, появившиеся из-за различного написанию слов. Использованы и строчные, и прописные буквы. Чтобы учесть такие дубликаты, все символы в строке приводят к нижнему регистру вызовом метода `str.lower()`. И потом проверим совпадают ли кол-во значений по категориям в 'education' со столбцом "education_id".	

In [29]:
# посчитаем количество дубликатов в столбце education
display(df['education'].value_counts())

среднее                13609
высшее                  4666
СРЕДНЕЕ                  764
Среднее                  700
неоконченное высшее      663
ВЫСШЕЕ                   270
Высшее                   266
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

In [30]:
df['education'] = df['education'].str.lower() # переводим все виды образования в нижний регистр
display(df['education'].value_counts()) # посмотрим, какой набор видов образования получился теперь

среднее                15073
высшее                  5202
неоконченное высшее      739
начальное                282
ученая степень             6
Name: education, dtype: int64

С дубликатами видов образования было 15, а без дубликатов всего лишь 5 (столько же, сколько в столбце `education_id` идентификаторов). 

In [31]:
df.duplicated().sum() #подсчет всех дубликатов

71

In [32]:
df = df.drop_duplicates().reset_index(drop=True)  # удаление  дубликатов (с удалением старых индексов и формированием новых)
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


In [33]:
df.duplicated().sum() # проверка на отсутствие дубликатов

0

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

In [36]:
education = df[['education_id', 'education']].drop_duplicates().reset_index(drop=True)
display(education)

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


In [37]:
family_status = df[['family_status_id', 'family_status']].drop_duplicates().reset_index(drop=True)
display(family_status)

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


In [38]:
df1=df.drop(['education', 'family_status'], axis = 1)
display(df1.head())

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу


### Шаг 2.6. Категоризация дохода.

In [40]:
df1['total_income_category'] = 'NaN'
df1.loc[df1['total_income'] <= 30000, ['total_income_category']] = ['E']
df1.loc[(30000 < df1['total_income']) & ( df1['total_income'] <= 50000), ['total_income_category']] = ['D']
df1.loc[(50000 < df1['total_income']) & ( df1['total_income'] <= 200000), ['total_income_category']] = ['C']
df1.loc[(200000 < df1['total_income']) & ( df1['total_income'] <= 1000000), ['total_income_category']] = ['B']
df1.loc[df1['total_income'] > 1000000, ['total_income_category']] = ['A']
print(df1['total_income_category'].value_counts())
display(df1.head())
print(df1.info())

C    15850

B     4987

D      347

A       25

E       22

Name: total_income_category, dtype: int64


,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C


<class 'pandas.core.frame.DataFrame'>

RangeIndex: 21231 entries, 0 to 21230

Data columns (total 11 columns):

 #   Column                 Non-Null Count  Dtype  

---  ------                 --------------  -----  

 0   children               21231 non-null  int64  

 1   days_employed          21231 non-null  float64

 2   dob_years              21231 non-null  int64  

 3   education_id           21231 non-null  int64  

 4   family_status_id       21231 non-null  int64  

 5   gender                 21231 non-null  object 

 6   income_type            21231 non-null  object 

 7   debt                   21231 non-null  int64  

 8   total_income           21231 non-null  int64  

 9   purpose                21231 non-null  object 

 10  total_income_category  21231 non-null  object 

dtypes: float64(1), int64(6), object(4)

memory usage: 1.8+ MB

None


### Шаг 2.7. Категоризация целей кредита.

In [41]:
# Для категоризации целей кредита выбрала функцию лемматизации. Импортируем библиотеку pymystem3
import json
from pymystem3 import Mystem
from collections import Counter
m = Mystem()

In [42]:
# Создадим функцию которая возвращает строку с ключевыми словами целей кредита
def lemma_pur(purpose):
    lemma = ' ' .join(m.lemmatize(purpose))
    return lemma

df1['purpose_word'] = df1['purpose'].apply(lemma_pur)

In [43]:
# Отобразим уникальные значения (цели кредита)
df1['purpose_word'].unique()

array(['покупка   жилье \n', 'приобретение   автомобиль \n',
       'дополнительный   образование \n', 'сыграть   свадьба \n',
       'операция   с   жилье \n', 'образование \n',
       'на   проведение   свадьба \n', 'покупка   жилье   для   семья \n',
       'покупка   недвижимость \n',
       'покупка   коммерческий   недвижимость \n',
       'покупка   жилой   недвижимость \n',
       'строительство   собственный   недвижимость \n', 'недвижимость \n',
       'строительство   недвижимость \n',
       'на   покупка   подержать   автомобиль \n',
       'на   покупка   свой   автомобиль \n',
       'операция   с   коммерческий   недвижимость \n',
       'строительство   жилой   недвижимость \n', 'жилье \n',
       'операция   со   свой   недвижимость \n', 'автомобиль \n',
       'заниматься   образование \n',
       'сделка   с   подержанный   автомобиль \n',
       'получение   образование \n', 'свадьба \n',
       'получение   дополнительный   образование \n',
       'покупка   свой 

In [44]:
# Создадим новый столбец (purpose_category) с категориями на основании результатов лемматизации
def purpose_category(list):
    if 'автомобиль' in list:
        return 'операции с автомобилем'
    if "образование" in list:
        return 'получение образования'
    if "свадьба" in list:
        return 'проведение свадьбы'
    if 'недвижимость' or 'жилье' or 'строительство' in list:
        return 'операции с недвижимостью'
    
df1['purpose_category'] = df1['purpose_word'].apply(purpose_category)

In [45]:
# После группировки цели кредита разделены на 4 группы
df1.drop(['purpose', 'purpose_word'], axis = 1)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,total_income_category,purpose_category
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,B,операции с недвижимостью
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,C,операции с автомобилем
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,C,операции с недвижимостью
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,B,получение образования
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,C,проведение свадьбы
...,...,...,...,...,...,...,...,...,...,...,...
21226,1,4529.316663,43,1,1,F,компаньон,0,224791,B,операции с недвижимостью
21227,0,343937.404131,67,1,0,F,пенсионер,0,155999,C,операции с автомобилем
21228,1,2113.346888,38,1,1,M,сотрудник,1,89672,C,операции с недвижимостью
21229,3,3112.481705,38,1,0,M,сотрудник,1,244093,B,операции с автомобилем


### Ответы на вопросы.

##### Вопрос 1: Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [46]:
df_pivot1 = df.pivot_table(index=['debt'], columns='children', values = 'family_status_id', aggfunc='count')
child0 = df_pivot1[0][1] / df_pivot1[0][0]
child1 = df_pivot1[1][1] / df_pivot1[1][0]
child2 = df_pivot1[2][1] / df_pivot1[2][0]
child3 = df_pivot1[3][1] / df_pivot1[3][0]
child4 = df_pivot1[4][1] / df_pivot1[4][0]

print("{0:.2f}% Нет детей".format(child0*100))
print("{0:.2f}% 1 Ребенок".format(child1*100))
print("{0:.2f}% 2 Ребенка".format(child2*100))
print("{0:.2f}% 3 Ребенка".format(child3*100))
print("{0:.2f}% 4 Ребенка".format(child4*100))

8.16% Нет детей

10.14% 1 Ребенок

10.51% 2 Ребенка

8.97% 3 Ребенка

10.81% 4 Ребенка


##### Вывод 1:

Чем больше количество детей, тем больше количество просроченных платежей. 

##### Вопрос 2: Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [47]:
df_pivot2 = df.pivot_table(index=['debt'], columns='family_status', values='dob_years', aggfunc='count')
no_family = df_pivot2['Не женат / не замужем'][1] / df_pivot2['Не женат / не замужем'][0]
divorce = df_pivot2['в разводе'][1] / df_pivot2['в разводе'][0]
widow = df_pivot2['вдовец / вдова'][1] / df_pivot2['вдовец / вдова'][0]
partner = df_pivot2['гражданский брак'][1] / df_pivot2['гражданский брак'][0]
family = df_pivot2['женат / замужем'][1] / df_pivot2['женат / замужем'][0]
print("{0:.2f}% Не женат / не замужем".format(no_family*100))
print("{0:.2f}% в разводе".format(divorce*100))
print("{0:.2f}% вдовец / вдова".format(widow*100))
print("{0:.2f}% гражданский брак".format(partner*100))
print("{0:.2f}% женат / замужем".format(family*100))


10.85% Не женат / не замужем

7.67% в разводе

7.01% вдовец / вдова

10.27% гражданский брак

8.18% женат / замужем


##### Вывод 2:

 Не женатые/ Не замужние или люди в гражданском браке реже выплачивают кредит в срок.

##### Вопрос 3: Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [48]:
df_pivot3 = df1.pivot_table(index=['debt'], columns='total_income_category', values='dob_years', aggfunc='count')
ud1 = df_pivot3['E'][1] / df_pivot3['E'][0]
ud2 = df_pivot3['D'][1] / df_pivot3['D'][0]
ud3 = df_pivot3['C'][1] / df_pivot3['C'][0]
ud4 = df_pivot3['B'][1] / df_pivot3['B'][0]
ud5 = df_pivot3['A'][1] / df_pivot3['A'][0]
print('{0:.2f}% E'.format(ud1*100))
print("{0:.2f}% D".format(ud2*100))
print("{0:.2f}% C".format(ud3*100))
print("{0:.2f}% B".format(ud4*100))
print("{0:.2f}% A".format(ud5*100))

10.00% E

6.44% D

9.28% C

7.62% B

8.70% A


##### Вывод 3:

Люди с наименьшим доходом чаще других групп выплачивают кредит в срок.

##### Вопрос 4: Как разные цели кредита влияют на его возврат в срок?

In [49]:
df_pivot4 =df1.pivot_table(index=['debt'], columns='purpose_category', values='dob_years', aggfunc='count')
ck1 = df_pivot4['операции с автомобилем'][1] / df_pivot4['операции с автомобилем'][0]
ck2 = df_pivot4['операции с недвижимостью'][1] / df_pivot4['операции с недвижимостью'][0]
ck3 = df_pivot4['получение образования'][1] / df_pivot4['получение образования'][0]
ck4 = df_pivot4['проведение свадьбы'][1] / df_pivot4['проведение свадьбы'][0]
print("{0:.2f}% операции с автомобилем".format(ck1*100))
print("{0:.2f}% операции с недвижимостью".format(ck2*100))
print("{0:.2f}% получение образования".format(ck3*100))
print("{0:.2f}% проведение свадьбы".format(ck4*100))

10.28% операции с автомобилем

7.83% операции с недвижимостью

10.25% получение образования

8.55% проведение свадьбы


##### Вывод 4:

Люди с большей вероятностью выплачивают кредит, если он взят на операции с недвижимостью.

## Общий вывод:

Общий вывод - семейный статус влияет на вероятность платежей по кредиту в срок - Не женатые/ Не замужние или люди в гражданском браке реже выплачивают кредит в срок. Чем больше детей, тем чаще платят не в срок, исключением являются пары с 3мя детьми. Возможно предоставленных данных не достаточно для корректкной оценки.